In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sagemaker boto3 awscli

In [ ]:
!aws s3 ls --profile asher

In [ ]:
local_dir = "./data"

In [7]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
role = "arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996" # get_execution_role() #
print("Role: ", role)
local_sess = LocalSession()  # for local session
# local_sess.config = {'local': {'local_code': True}}

# sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1")) #name of sso session remove profile name in sagemaker studio , profile_name="asher"
# #for remote session
# settings = dict(
#     sagemaker_session=sess,
#     role=role,
#     instance_type="ml.t3.large",
#     dependencies='./requirements.txt',
# )

bucket_name = sess.default_bucket()
print(bucket_name)

Role:  arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996
sagemaker-ap-southeast-1-345594598345


Uploaded training, testing and validation datasets to S3 bucket

In [ ]:
# !aws s3 cp ./data s3://{bucket_name}/prostate158 --recursive --profile asher
# print("Done!")

Upload weights of pre-trained UNet model

In [ ]:
# !aws s3 cp models/tumor.pt s3://{bucket_name}/models/tumor.pt --profile asher

In [4]:
if bucket_name:
    train_channel = f"s3://{bucket_name}/prostate158/prostate158_train/train"
    test_channel = f"s3://{bucket_name}/prostate158/prostate158_test/test"
    model_channel = f"s3://{bucket_name}/models/tumor.pt"
inputs = {"train": train_channel, "test": test_channel, "model": model_channel}
print(inputs)

{'train': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_train/train', 'test': 's3://sagemaker-ap-southeast-1-345594598345/prostate158/prostate158_test/test', 'model': 's3://sagemaker-ap-southeast-1-345594598345/models/tumor.pt'}


In [5]:
!pip install munch

  Using cached munch-4.0.0-py2.py3-none-any.whl.metadata (5.9 kB)
Using cached munch-4.0.0-py2.py3-none-any.whl (9.9 kB)


In [5]:
import yaml
from pathlib import Path
from munch import Munch
# config = yaml.safe_load(Path("tumor.yaml").read_text())
config = Munch.fromDict(yaml.safe_load(Path("tumor.yaml").read_text()))


In [12]:
%%time
from sagemaker.pytorch.estimator import PyTorch
import os

image = "763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu124-ubuntu22.04-sagemaker"

def run_training():
    estimator = PyTorch(
        image_uri=image,
        entry_point="train_gpt.py",
        source_dir=".",
        role=role,
        region="ap-southeast-1",
        # instance_type="ml.g4dn.2xlarge", # ml.g5.xlarge ran out of memory halfway through 2nd epoch (free tier instance)
        instance_type='local',
        instance_count=1,
        framework_version="2.6.0",
        py_version="py312",
        enable_spot_training=True,  # for cost savings
        # max_run=3600,
        input_mode="File",
        sagemaker_session=local_sess,
    )
    inputs={
        "training": train_channel,
        "model": model_channel,
        "training_config": os.environ.get("SM_CHANNEL_TRAINING_CONFIG", "file://tumor.yaml"),
    }
    estimator.fit(inputs)

run_training()

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-07-15-09-16-03-273
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not inst

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/IPython/core/magics/execution.py:1375 in time            │
│                                                                                                  │
│   1372 │   │   │   │   # multi-line %%time case                                                  │
│   1373 │   │   │   │   if expr_val is not None:                                                  │
│   1374 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1375 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1376 │   │   │   except Exception:                                                             │
│   1377 │   │   │   │   self.shell.showtraceback()                                                │
│   1378 │   │   │   │   return                                                                    │
│ in <module>:30                                                                                   │
│ in run_training:28                                                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:167 in wrapper  │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:138 in wrapper  │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                    

In [12]:
# !aws s3 cp s3://sagemaker-ap-southeast-1-345594598345/pytorch-training-2025-07-15-08-01-51-392/output/model.tar.gz .
!tar -xvzf model.tar.gz


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
network_tumor_1_key_metric=0.0066.pt
